# Imports

In [1]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
pd.set_option('display.max_columns', None)

output_folder = 'raw_dataset2' # write the cleaned processed files here

if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

# Population Estimation

## Download

In [2]:
url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/asrh/CC-EST2020-AGESEX-ALL.csv'
response = requests.get(url)
file_object = io.StringIO(response.content.decode('latin1')) # contains letters undecodable for utf-8
df = pd.read_csv(file_object, low_memory=False) # low_memory=False because features have mixed types

# in case the download by code is not working
# df = pd.read_csv(f'{output_folder}/CC-EST2020-AGESEX-ALL.csv', encoding='latin1', low_memory=False)
df.head(3)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,1,1,Alabama,Autauga County,1,54571,26569,28002,3579,1866,1713,7418,3747,3671,3616,1842,1774,4617,2335,2282,41804,20046,21758,39958,19114,20844,22100,10867,11233,14730,7115,7615,14065,6843,7222,6546,2821,3725,3579,1866,1713,3991,2001,1990,4290,2171,2119,4290,2213,2077,3080,1539,1541,3157,1543,1614,3330,1594,1736,4157,2004,2153,4086,1974,2112,4332,2174,2158,3873,1866,2007,3083,1524,1559,2777,1279,1498,2277,1014,1263,1736,807,929,1251,546,705,731,295,436,551,159,392,37,35.9,37.9
1,50,1,1,Alabama,Autauga County,2,54582,26576,28006,3582,1868,1714,7425,3752,3673,3617,1842,1775,4617,2335,2282,41804,20045,21759,39958,19114,20844,22103,10867,11236,14732,7115,7617,14067,6845,7222,6542,2819,3723,3582,1868,1714,3994,2004,1990,4294,2173,2121,4291,2213,2078,3080,1539,1541,3160,1545,1615,3329,1593,1736,4157,2004,2153,4086,1973,2113,4333,2176,2157,3874,1866,2008,3083,1524,1559,2777,1279,1498,2277,1014,1263,1734,806,928,1250,546,704,730,294,436,551,159,392,37,35.9,37.8
2,50,1,1,Alabama,Autauga County,3,54761,26667,28094,3575,1862,1713,7400,3720,3680,3565,1819,1746,4670,2362,2308,42038,20190,21848,40221,19266,20955,22194,10939,11255,14815,7185,7630,14137,6872,7265,6599,2847,3752,3575,1862,1713,3964,1984,1980,4292,2163,2129,4228,2181,2047,3151,1573,1578,3188,1573,1615,3369,1614,1755,4142,2000,2142,4116,1998,2118,4321,2163,2158,3908,1884,2024,3119,1533,1586,2789,1292,1497,2289,1018,1271,1752,814,938,1259,552,707,743,299,444,556,164,392,37.1,36,37.9


## Preprocessing

In [3]:
df = df[df['YEAR']==13] # category 13 is for year 2020 estimation
df.drop(columns=['SUMLEV', 'YEAR'], inplace=True)
df['Name'] = df['STNAME'] + ', ' + df['CTYNAME'].apply(lambda x: x.replace(' County', ''))
df['FIPS'] = df['STATE'] * 1000 + df['COUNTY']

identity_columns = ['STATE','COUNTY','FIPS','STNAME','CTYNAME','Name']
other_columns = [col for col in df.columns if col not in identity_columns]
df = df[identity_columns+other_columns]
df.head(3)

,STATE,COUNTY,FIPS,STNAME,CTYNAME,Name,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
12,1,1,1001,Alabama,Autauga County,"Alabama, Autauga",56130,27231,28899,3346,1739,1607,6566,3366,3200,3114,1553,1561,4416,2267,2149,44657,21353,23304,43104,20573,22531,21567,10606,10961,14798,7166,7632,14848,7161,7687,9042,3979,5063,3346,1739,1607,3494,1785,1709,3833,1961,1872,3622,1825,1797,3147,1615,1532,3822,1893,1929,3636,1756,1880,3743,1818,1925,3597,1699,1898,3820,1844,1976,3675,1811,1864,3912,1915,1997,3441,1591,1850,2790,1351,1439,2386,1032,1354,1796,749,1047,1142,494,648,928,353,575,39.3,37.8,40.5
26,1,3,1003,Alabama,Baldwin County,"Alabama, Baldwin",227989,110420,117569,12092,6319,5773,24636,12572,12064,11536,5787,5749,16067,8055,8012,185376,88563,96813,179725,85742,93983,77312,38054,39258,52651,25693,26958,62077,29384,32693,48930,22610,26320,12092,6319,5773,13356,6874,6482,14222,7179,7043,13371,6684,6687,11290,5677,5613,12517,6254,6263,12865,6256,6609,13524,6579,6945,13745,6604,7141,14328,6941,7387,14519,6963,7556,16420,7756,8664,16810,7724,9086,15519,7146,8373,13806,6532,7274,9457,4503,4954,5607,2554,3053,4541,1875,2666,43.8,42.5,45.2
40,1,5,1005,Alabama,Barbour County,"Alabama, Barbour",24652,13018,11634,1315,673,642,2574,1275,1299,1164,593,571,1992,1162,830,20183,10777,9406,19599,10477,9122,9300,5525,3775,6441,3917,2524,6188,3234,2954,4978,2164,2814,1315,673,642,1351,659,692,1520,763,757,1394,726,668,1465,882,583,1762,1113,649,1648,1004,644,1559,899,660,1472,901,571,1552,869,683,1467,795,672,1659,827,832,1510,743,767,1499,689,810,1406,619,787,971,439,532,587,245,342,515,172,343,40.9,38.9,44.5


In [4]:
# https://www.census.gov/programs-surveys/geography/technical-documentation/county-changes/2010.html
if df[df['FIPS'].isin([2063, 2066])].shape[0] == 2:
    values = df[df['FIPS']==2063][other_columns].values[0] + df[df['FIPS']==2066][other_columns].values[0]
    df.loc[len(df.index)] = [2, 261, 2261, 'Alaska', 'Valdez-Cordova', 'Valdez-Cordova County'] + list(values)
    df = df[~df['FIPS'].isin([2063, 2066])]

    df = df.sort_values(by=['FIPS'])

## Dump

In [6]:
df.to_csv(f'{output_folder}/Population.csv', index=False)

# Age Groups

## Preprocess

The subgroups are defined by the [ground truth released by CDC](https://data.cdc.gov/Public-Health-Surveillance/COVID-19-Weekly-Cases-and-Deaths-by-Age-Race-Ethni/hrdz-jaxc) on COVID-19 infection spread by US counties. This allows us to compare the results with the ground truth.

In [13]:
df = pd.read_csv(f'{output_folder}/Population.csv')

df['UNDER5'] = df['UNDER5_TOT']
df['AGE517'] = df['AGE513_TOT'] + df['AGE1417_TOT']
df['AGE1829'] = df['AGE1824_TOT'] + df['AGE2529_TOT']
df['AGE3039'] = df['AGE3034_TOT'] + df['AGE3539_TOT']
df['AGE4049'] = df['AGE4044_TOT'] + df['AGE4549_TOT']

df['AGE5064'] = df['AGE5054_TOT'] + df['AGE5559_TOT']+ df['AGE6064_TOT']
df['AGE6574'] = df['AGE6569_TOT'] + df['AGE7074_TOT']
df['AGE75PLUS'] = df['AGE7579_TOT'] + df['AGE8084_TOT'] + df['AGE85PLUS_TOT']

In [15]:
selected_age_groups = [
    'UNDER5', 'AGE517', 'AGE1829', 'AGE3039', 'AGE4049',
    'AGE5064', 'AGE6574', 'AGE75PLUS'
]
for age_group in selected_age_groups:
    df[age_group] /= df['POPESTIMATE']
    
df = df[['FIPS'] + selected_age_groups]
df.head(3)

,FIPS,UNDER5,AGE517,AGE1829,AGE3039,AGE4049,AGE5064,AGE6574,AGE75PLUS
0,1001,0.059612,0.172457,0.146766,0.131463,0.132140,0.196472,0.092215,0.068876
1,1003,0.053038,0.158657,0.125374,0.115747,0.123133,0.209436,0.128625,0.085991
2,1005,0.053343,0.151631,0.152280,0.130091,0.122668,0.188058,0.117840,0.084091


## Dump

In [17]:
df.round(6).to_csv(f'{output_folder}/Age Groups.csv', index=False)